In [1]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from datasets import load_dataset, Audio

# Cấu hình
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "HoaDoan1710/checkpoint-whisper-14525"  # Model của bạn đã fine-tune
PROCESSOR_PATH = "hkab/whisper-base-vietnamese-finetuned"  # Processor từ model tiếng Việt
# PROCESSOR_PATH = "openai/whisper-medium", language="Vietnamese",task="transcribe"

# Load processor và model
processor = WhisperProcessor.from_pretrained(PROCESSOR_PATH)  # Sử dụng processor tiếng Việt đã tối ưu
model = WhisperForConditionalGeneration.from_pretrained(MODEL_PATH).to(DEVICE)
model.config.forced_decoder_ids = None

# Load dataset và lấy 3 mẫu
dataset = load_dataset("google/fleurs", "vi_vn", split="train", trust_remote_code=True)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
samples = dataset.select(range(3))  # Lấy 3 mẫu đầu tiên

# Hàm transcribe
def transcribe(audio_array):
    inputs = processor(audio_array, sampling_rate=16_000, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        predicted_ids = model.generate(**inputs)
    return processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# Kiểm tra kết quả
for i, sample in enumerate(samples):
    audio = sample["audio"]["array"]
    ground_truth = sample["transcription"]
    prediction = transcribe(audio)
    
    print(f"\n📌 Sample {i+1}:")
    print(f"✅ Ground Truth: {ground_truth}")
    print(f"🤖 Prediction_base_model: {prediction}")
    print("-" * 50)

2025-05-14 15:09:07.622762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747235347.831634      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747235347.886728      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



📌 Sample 1:
✅ Ground Truth: theo sau quá trình tố tụng này hjr-3 sẽ được xem xét lại bởi cơ quan lập pháp được bầu cử tiếp theo trong năm 2015 hoặc 2016 để tiếp tục quá trình xử lý
🤖 Prediction_base_model: theo sau quá trình tố tuần này hát chí e rờ ba sẽ được xem xét lại bởi cơ quan lập pháp được bầu cử tiếp theo trong năm hai ngàn không trăm mười lăm hoặc hai ngàn không trăm mười giáo để tiếp tục quá trình xử lý
--------------------------------------------------

📌 Sample 2:
✅ Ground Truth: phát minh bánh xe nan hoa làm cho những cỗ xe ngựa chiến của người assyria nhẹ hơn nhanh hơn và sẵn sàng hơn để đánh bại các binh lính và cỗ xe khác
🤖 Prediction_base_model: phát minh bánh xe năng hoa làm cho những cổ xe n ngựa chiến của người a si ria nhỏ hơn nhanh hơn và sẵn sàng hơn để đến mọi cách binh lính và cổ xe khác
--------------------------------------------------

📌 Sample 3:
✅ Ground Truth: karno là một thầy giáo dạy tiếng anh nổi tiếng nhưng gây ra nhiều tranh cãi là người đã giảng 